In [1]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
# import matplotlib.pyplot as plt 
import time
# import numpy as np
# import scipy as sp
import re
from collections import Counter
import random

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
# from nltk.corpus import stopwords
# s=stopwords.words('english') + ['']

# Machine learning
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC, SVC
# from sklearn.pipeline import Pipeline
# from sklearn import metrics

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [2]:
query = session.query(Article).filter(or_(Article.journal_ref.like('Nature%'),
                                          Article.journal_ref.like('Nat.%'),
                                          Article.journal_ref.like('Science%')))
abstract_all = [x.abstract for x in query.all()]
# titleNatureScience = [x.title for x in query.all()]
session.close_all()

In [3]:
abstract_train = abstract_all
print len(abstract_train)

3384


CountVectorizer might do a cleaner job.

In [4]:
class AbstractWriter:
    '''This class will take a specific n-gram and learn to write abstracts from a few words.'''
    _data = dict()
    _abstracts = []
    
    def __init__(self, ngram, randomize=False, seed=None, maxWords=200):
        self.ngram = ngram
        self.randomize = randomize
        self.seed = seed
        self.maxWords = maxWords
        if seed is not None:
            random.seed(seed)

    
    def fit(self, train):
        data_list = []
        for item in train:
            ind = 0
            textlist = self.__cleanup_text(item)
            self._abstracts.append(' '.join(textlist))
            while ind + self.ngram - 1 < len(textlist):
                currentTuple = tuple(textlist[ind:(ind+self.ngram-1)])
                nextWord = textlist[ind+self.ngram-1]
                data_list.append((tuple(textlist[ind:(ind+self.ngram-1)]),
                                        textlist[ind+self.ngram-1]))
                ind += 1
        data_list.sort()
        
        # Now that the list is sorted, convert it into a dictionary.
        # Since the list is sorted, each subsuquent entry is either new or a continuation of the previous one.

        previousKey = None
        for val in data_list:
            currentKey = val[0]
            if currentKey == previousKey:
                self._data[currentKey].append(val[1])
            else:
                self._data[currentKey] = [val[1]]
            previousKey = val[0]

    def find_next_word(self, phrase):
        '''phrase is a tuple of (typ. 3) words. Returns the next word.'''
        # Since data is sorted, it should be possible to search quickly. I'm not doing this properly.
        if phrase in self._data.keys():
            solution = self._data[phrase]
            
            # If there are more than one result, we either return the most popular option or
            # a random answer weighted by popularity.
            if self.randomize:
                return random.choice(solution)
            else:
                count = Counter(solution)
                return count.most_common(1)[0][0]
        else:
            return None
        
    def write_abstract(self, beginning):
        abstract = list(beginning)
        nextWord = ''
        ind = len(beginning) - self.ngram + 1
        while (nextWord is not None) and (ind < self.maxWords):
            nextWord = self.find_next_word(tuple(abstract[ind:(ind+self.ngram)]))
            if nextWord is None:
                break
            else:
                abstract.append(nextWord)
                ind += 1
        return ' '.join(abstract)
    
    def find_similar(self, abstract):
        '''Have I accidentally plagarized one of the articles?
        With too few choices, we will likely just copy an existing article.
        Returns the longest chunk highlighted in an existing article.'''
        first = 0
        last = 1
        abort = False
        longestChunk = ''
        longestChunkLength = 0

        totalLength = len(abstract.split())
        while first + longestChunkLength < totalLength:
            chunk = ' '.join(abstract.split()[first:last])
            if any(x for x in self._abstracts if chunk in x):
                if last - first + 1 > longestChunkLength:
                    longestChunk = chunk
                    longestChunkLength = last - first + 1
                last += 1
            else:
                first += 1
                last = first + 1
        
        best_abstract = (x for x in self._abstracts if longestChunk in x).next()
        return re.sub('(' + longestChunk + ')', '[[[' + longestChunk + ']]]', best_abstract)


    def __cleanup_text(self, text):
        # re from kaggle https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-wordsi
        # Remove punctuation. Keep periods.
        # 
        # New trick: replace numbers with ##, which we will later replace with different numbers.
        pattern1 = re.compile("[0-9]")
        pattern2 = re.compile("[^a-zA-Z#.]")
        # The previous pattern could make multiple spaces. Turn multiple spaces into a single space.
        pattern3 = re.compile("(\s{2,})")
        return pattern3.sub(" ",
                            pattern2.sub(" ",
                                         pattern1.sub("#",
                                                      text.lstrip().rstrip()))).split(' ')

"In this Letter, we" is a popular phrase. Let's make a bunch of abstracts that start that way, and find if they accidentally plagarize anything.

In [5]:
def InThisLetterWe(ngram):
    writer = AbstractWriter(ngram=ngram, randomize=True, seed=42)
    writer.fit(abstract_train)
    basephrase = ['In','this', 'Letter', 'we']
#     if ngram == 6:
#         writer5 = AbstractWriter(ngram=5, randomize=True, seed=42)
#         phrase += random.choice(['report'], ['show'])
    if ngram == 6:
        writer5 = AbstractWriter(ngram=5, randomize=True, seed=42)
    for i in range(0,5):
        if ngram == 6:
            phrase = basephrase + [writer5.find_next_word(tuple(basephrase))]
        else:
            phrase = basephrase
        currentAbstract = writer.write_abstract(phrase)
        print 'New abstract: ' + currentAbstract
        print ''
        print 'Existing abstract: ' + writer.find_similar(currentAbstract)
        print ''
    return writer

def WriteAbstract(ngram):
    writer = AbstractWriter(ngram=ngram, randomize=True, seed=42)
    writer.fit(abstract_train)
    for i in range(0,5):
        phrase = random.choice(writer._abstracts).split()[0:(ngram-1)]
        currentAbstract = writer.write_abstract(tuple(phrase))
        print 'New abstract: ' + currentAbstract
        print ''
        print 'Existing abstract: ' + writer.find_similar(currentAbstract)
        print ''
    return writer

In [6]:
WriteAbstract(4)

New abstract: Quantum tunneling constitutes one of the most notorious non Fermi liquid and Fermi liquid are two distinct parallel sequences of blue stragglers in many clusters are bimodal with a dominant central peak this has been shown that in these objects traces shocks in the ejecta. A few enhanced Fe features are basically aligned along the nanotube axis. While the origin of such vortices is presented in terms of alternating equilibrium and non equilibrium quantum thermodynamics. In the absence of dissipation stable composite objects are usually formed via attractive forces which allow the constituents to lower their energy by binding together. Repulsive forces separate particles in free space. We directly image such individual adatoms along with carbon chains and vacancies and investigate their dynamics via the dielectric function epsilon nu . Here we present a substantial improvement by combining a very accurate approximation to a deeper level theory which reconciles the determin

<__main__.AbstractWriter instance at 0x103c705f0>

In [7]:
WriteAbstract(5)

New abstract: Quantum mechanics the theory describing the fundamental workings of nature is famously counterintuitive it predicts that a particle can be in two places at the same time the links between the updated vertices and all other vertices are drawn anew with a fitness dependent connection probability. We show analytically and numerically that the system self organizes to a non trivial state that differs from what is obtained when the two processes are decoupled. A power law decay of dynamical and topological quantities above a threshold emerges spontaneously as well as a Lifshitz transition with a change of Fermi surface topology. Here we report high resolution radio imaging of the gamma ray emitting nova V### Mon. We find that its ejecta were shaped by the motion of the binary system some gas was expelled rapidly along the poles as a wind from the disk and sometimes a collimated jet. But these predictions have hitherto not been supported by observations. Here we report an antag

<__main__.AbstractWriter instance at 0x10d86a4d0>

In [8]:
WriteAbstract(6)

New abstract: Solar cells based on the organic inorganic tri halide perovskite family of materials have shown remarkable progress recently offering the prospect of low cost solar energy from devices that are very simple to process. Fundamental to understanding the operation of these devices is the exciton binding energy which has proved both difficult to measure directly and controversial. We demonstrate that by using very high magnetic fields it is possible to make an accurate and direct spectroscopic measurement of the exciton binding energy which has proved both difficult to measure directly and controversial. We demonstrate that by using very high magnetic fields it is possible to make an accurate and direct spectroscopic measurement of the exciton binding energy which we find to be strikingly different from the predictions of single particle theory. The states are mainly determined by Coulomb interactions and appear only when quantization of kinetic energy limits the screening abi

<__main__.AbstractWriter instance at 0x11838c680>